In [1]:
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import json
import lxml
import pandas as pd
import pyterrier as pt
import string



In [2]:
!rm -rf ./index

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/caleb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/caleb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
input_file = "Answers.json"
query_file = "topics_2.json"
qrel_file = "qrel_1.tsv"

with open(input_file,'r') as file:
    answers = json.load(file)

with open(query_file,'r') as file:
    queries = json.load(file)

In [5]:
#very simple preprocessing, parses inputs with lxml parser as default python parser removed
# way too much of the anwswer text.
def process_input(data):
    for item in data:
        text = item['Text']
        soup = BeautifulSoup(text, "lxml")
        text = soup.get_text()
        item['Text'] = text

In [6]:
process_input(answers)

In [7]:
index_path = "./index"

In [8]:
#define format of incoming data
columns = ['Id','Text','Score']

In [9]:
#reformats answers to be in terrier compatible format
answer_frame = pd.DataFrame(answers, columns=columns)
answer_frame = answer_frame.rename(columns={'Id': 'docno','Text':'text'})

In [10]:
answer_frame

,docno,text,Score
0,2,Maybe these instructions will help you to get ...,41
1,10,You could use freenx instead of vnc. Freenx tr...,12
2,12,"Yes, X forwarding over ssh is a beautiful thin...",22
3,18,You can use the alternative CD (instead of Ubu...,7
4,19,I've always helped identified people's needs a...,8
...,...,...,...
523723,1509129,"If anyone struggling with this, copy the conte...",1
523724,1509131,Edit the file /etc/systemd/sleep.conf with the...,0
523725,1509140,Had a similar issue withAndroid Smartphone: Hu...,1
523726,1509147,"As described in another answer, the solusion i...",0


In [11]:
pd_indexer = pt.terrier.IterDictIndexer(index_path).index(answer_frame.to_dict(orient='records'))

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


16:31:04.229 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (16123) - further warnings are suppressed
16:31:29.096 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 4 empty documents


In [12]:
index = pt.IndexFactory.of(pd_indexer)

In [13]:
print(index.getCollectionStatistics().toString())

Number of documents: 523728
Number of terms: 621208
Number of postings: 19911657
Number of fields: 1
Number of tokens: 31765345
Field names: [text]
Positions:   false



In [18]:
"""def process_queries(queries):
    #this converts the raw json querries into [qid: 'num',query:[token,token,token]]
    processed_queries = pd.DataFrame(columns=['qid', 'query'])

    for index, row in queries.iterrows():
        qid, title = row['Id'], row['Title']
        #THIS IS SOOOO IMPORTANT HOLY CRAP WHY DOES THE "?" CHAR CRASH TERRIER?!?!?!?!?
        title = title.translate(str.maketrans('', '', string.punctuation))
        new_row = {'qid': qid, 'query': title}
        new_row_df = pd.DataFrame([new_row])
        processed_queries = pd.concat([processed_queries, new_row_df], ignore_index=True)
        
    return processed_queries
    """ #deprecated this should work but doesnt...

In [14]:
#processes the queires via the provided class example.
#further expienments should change how we do this
def process_queries(queries):
    lst_q = []
    for item in queries:
        lst_q.append([item['Id'],
        item['Title'].translate(str.maketrans('', '', string.punctuation))])
    processed_queries = pd.DataFrame(lst_q, columns=["qid", "query"])
    return processed_queries

In [15]:
queries = json.load(open(query_file))
processed_queries = process_queries(queries)

In [16]:
#define the retreivers for all the models used

bm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "BM25"},verbose=True,num_results=1000)
dfiz_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIZ"},verbose=True,num_results=1000)
dfic_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIC"},verbose=True,num_results=1000)
hiemstralm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "Hiemstra_LM"},verbose=True,num_results=1000)
dfree_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFRee"},verbose=True,num_results=1000)
dfreeKlim_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFReeKLIM"},verbose=True,num_results=1000)
tf_retr = pt.terrier.Retriever(index,controls = {"wmodel": "TF_IDF"},verbose=True,num_results=1000)

In [23]:
#Testing for two models to make sure things are properly working
bm_res = bm_retr.transform(processed_queries)
dfiz_res = dfiz_retr.transform(processed_queries)
dfic_res = dfic_retr.transform(processed_queries)
hiem_res = hiemstralm_retr.transform(processed_queries)
dfree_res = dfree_retr.transform(processed_queries)
dfreeKlim_res = dfreeKlim_retr.transform(processed_queries)
tf_res = tf_retr.transform(processed_queries)

TerrierRetr(TF_IDF): 100%|██████████| 1000/1000 [00:16<00:00, 59.33q/s]


In [24]:
bm_res = pd.DataFrame(bm_res)
dfiz_res = pd.DataFrame(dfiz_res)
dfic_res = pd.DataFrame(dfic_res)
hiem_res = pd.DataFrame(hiem_res)
dfree_res = pd.DataFrame(dfree_res)
dfreeKlim_res = pd.DataFrame(dfreeKlim_res)
tf_res = pd.DataFrame(tf_res)

bm_res.to_csv('bm_results_topic2.tsv', sep='\t', index=False)
dfiz_res.to_csv('dfiz_results_topic2.tsv', sep='\t', index=False)
dfic_res.to_csv('dfic_results_topic2.tsv', sep='\t', index=False)
hiem_res.to_csv('hiem_results_topic2.tsv', sep='\t', index=False)
dfree_res.to_csv('dfree_results_topic2.tsv', sep='\t', index=False)
dfreeKlim_res.to_csv('dfrklim_results_topic2.tsv', sep='\t', index=False)
tf_res.to_csv('tf_results_topic2.tsv', sep='\t', index=False)

In [17]:
#evaluation
from pyterrier.measures import *
qrels = pt.io.read_qrels(qrel_file)

In [18]:
metrics = ["ndcg", "P_5", "P_10", "P", "map", "bpref","recip_rank"]

In [ ]:
#defines the expirment to run using chosen metrics and excluding queries not in the qrel
res = pt.Experiment(
    [bm_retr,dfiz_retr,dfic_retr,hiemstralm_retr,dfree_retr,dfreeKlim_retr,tf_retr],
    processed_queries,
    qrels,
    eval_metrics=metrics,
    names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"],
    save_dir="./",
    filter_by_qrels=False,
    filter_by_topics=False,
    test='t',
    baseline=6
)

In [ ]:
print(res)

In [122]:
#formatting results into a dataframe for export
names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"]
res_df = pd.DataFrame(res)

In [107]:
res_df.to_csv('results.tsv', sep='\t', index=False)

In [ ]:
#runs a per query expirement for top 3 models
per_query = pt.Experiment(
    [bm_retr,tf_retr,dfree_retr],
    processed_queries,
    qrels,
    eval_metrics=["ndcg", "P_5", "bpref","recip_rank"],
    filter_by_qrels=True,
    perquery=True,
    names=["BM25","TF-IDF","DFreeKlim"]
    )

In [130]:
per_query.to_csv('per_q.tsv',sep='\t', index=False)

In [ ]:
#Ad-hoc querry
ad_hoc = "best way to monitor processes"
top_k = 5
top_k_bm = bm_retr.search(ad_hoc).head(top_k)
top_k_df = dfreeKlim_retr.search(ad_hoc).head(top_k)


In [31]:
#for in editor viewing of result information.
df_results = pd.read_csv("results/topic_1_DFreeKlim.res/DFreeKlim.res", sep=' ')
tf_results = pd.read_csv("results/topic_1_TF-IDF.res/TF-IDF.res", sep=' ')
bm_results = pd.read_csv("results/topic_1_BM25.res/BM25.res", sep=' ')